In [ ]:
!nvidia-smi -L

In [ ]:
!tree -d ../../images/png

In [ ]:
!pip3 install numpy -q
!pip3 install torch -q
!pip3 install torchvision -q
!pip3 install matplotlib -q
!pip3 install tqdm -q
!pip3 install ipywidgets -q
!pip3 install opencv-python -q

In [ ]:
import os
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image

In [ ]:
# Set random seed for reproducibility
# manualSeed = 42
manualSeed = random.randint(1, 10000) # use if you want new results
random.seed(manualSeed)
torch.manual_seed(manualSeed)
print("Random Seed: ", manualSeed)

In [ ]:
class DCGAN():

    def __init__(
        self,                   
        dataroot,               # Root directory for dataset
        logfolder,
        workers = 2,            # Number of workers for dataloader
        batch_size = 128,       # Batch size during training
        image_size = 64,        # Spatial size of training images. All images will be resized to this size using a transformer.
        nc = 3,                 # Number of channels in the training images. For color images this is 3
        nz = 100,               # Size of z latent vector (i.e. size of generator input)
        ngf = 64,               # Size of feature maps in generator
        ndf = 64,               # Size of feature maps in discriminator
        num_epochs = 50,        # Number of training epochs
        lr = 0.0002,            # Learning rate for optimizers
        beta1 = 0.5,            # Beta1 hyperparam for Adam optimizers
        ngpu = 1                # Number of GPUs available. Use 0 for CPU mode.
        ):
        # Hyperparameters etc.
        self.logfolder = logfolder
        self.device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
        self.LEARNING_RATE = lr  # could also use two lrs, one for gen and one for disc
        self.BATCH_SIZE = batch_size
        self.IMAGE_SIZE = image_size
        self.CHANNELS_IMG = nc
        self.NOISE_DIM = nz
        self.NUM_EPOCHS = num_epochs
        self.FEATURES_DISC = ndf
        self.FEATURES_GEN = ngf
        self.BETA = beta1

        self.transform = transforms.Compose(
            [
                transforms.Resize((self.IMAGE_SIZE, self.IMAGE_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(
                    [0.5 for _ in range(self.CHANNELS_IMG)], [0.5 for _ in range(self.CHANNELS_IMG)]
                ),
            ]
        )

        # comment mnist above and uncomment below if train on CelebA
        self.dataset = datasets.ImageFolder(root=dataroot, transform=self.transform)
        self.dataloader = DataLoader(self.dataset, batch_size=self.BATCH_SIZE, shuffle=True, num_workers=workers)
        self.gen = self.Generator(self.NOISE_DIM, self.CHANNELS_IMG, self.FEATURES_GEN).to(self.device)
        self.disc = self.Discriminator(self.CHANNELS_IMG, self.FEATURES_DISC).to(self.device)
        if (self.device.type == 'cuda') and (ngpu > 1):
            self.gen = nn.DataParallel(self.gen, list(range(ngpu)))
            self.disc = nn.DataParallel(self.disc, list(range(ngpu)))
        self.gen.apply(self.weights_init)
        self.disc.apply(self.weights_init)

        # Initialize BCELoss function
        self.criterion = nn.BCELoss()
        # Create batch of latent vectors that we will use to visualize the progression of the generator
        self.fixed_noise = torch.randn(64, self.NOISE_DIM, 1, 1, device=self.device)
        # Establish convention for real and fake labels during training
        self.real_label = 1.
        self.fake_label = 0.
        # Setup Adam optimizers for both G and D
        self.opt_gen = optim.Adam(self.gen.parameters(), lr=self.LEARNING_RATE, betas=(self.BETA, 0.999))
        self.opt_disc = optim.Adam(self.disc.parameters(), lr=self.LEARNING_RATE, betas=(self.BETA, 0.999))


    def weights_init(self, model):
        # Initializes weights according to the DCGAN paper
        for m in model.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.normal_(m.weight.data, 0.0, 0.02)
            elif isinstance(m, (nn.BatchNorm2d)):
                nn.init.normal_(m.weight.data, 0.0, 0.02)
                nn.init.constant_(m.bias.data, 0)

    def train(self):

        # Training Loop

        # Lists to keep track of progress
        img_list = []
        G_losses = []
        D_losses = []

        toPrintEpoch = list(range(0, self.NUM_EPOCHS, max((self.NUM_EPOCHS // 100), 1)))

        # For each epoch
        for epoch in tqdm(range(self.NUM_EPOCHS)):
            # For each batch in the dataloader
            for data in self.dataloader:

                ############################
                # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
                ###########################
                ## Train with all-real batch
                self.disc.zero_grad()
                # Format batch
                real_cpu = data[0].to(self.device)
                b_size = real_cpu.size(0)
                label = torch.full((b_size,), self.real_label, dtype=torch.float, device=self.device)
                # Forward pass real batch through D
                output = self.disc(real_cpu).view(-1)
                # Calculate loss on all-real batch
                errD_real = self.criterion(output, label)
                # Calculate gradients for D in backward pass
                errD_real.backward()
                D_x = output.mean().item()

                ## Train with all-fake batch
                # Generate batch of latent vectors
                noise = torch.randn(b_size, self.NOISE_DIM, 1, 1, device=self.device)
                # Generate fake image batch with G
                fake = self.gen(noise)
                label.fill_(self.fake_label)
                # Classify all fake batch with D
                output = self.disc(fake.detach()).view(-1)
                # Calculate D's loss on the all-fake batch
                errD_fake = self.criterion(output, label)
                # Calculate the gradients for this batch, accumulated (summed) with previous gradients
                errD_fake.backward()
                D_G_z1 = output.mean().item()
                # Compute error of D as sum over the fake and the real batches
                errD = errD_real + errD_fake
                # Update D
                self.opt_disc.step()

                ############################
                # (2) Update G network: maximize log(D(G(z)))
                ###########################
                self.gen.zero_grad()
                label.fill_(self.real_label)  # fake labels are real for generator cost
                # Since we just updated D, perform another forward pass of all-fake batch through D
                output = self.disc(fake).view(-1)
                # Calculate G's loss based on this output
                errG = self.criterion(output, label)
                # Calculate gradients for G
                errG.backward()
                D_G_z2 = output.mean().item()
                # Update G
                self.opt_gen.step()

                # Save Losses for plotting later
                G_losses.append(errG.item())
                D_losses.append(errD.item())
            
            if (epoch in toPrintEpoch):
                # Check how the generator is doing by saving G's output on fixed_noise
                with torch.no_grad():
                    no_grad_fake = self.gen(self.fixed_noise).detach().cpu()

                img_grid_fake = vutils.make_grid(no_grad_fake, padding=2, normalize=True)
                
                plt.figure(figsize=(15,15))
                plt.axis("off")
                plt.imshow(np.transpose(img_grid_fake,(1,2,0)))
                plt.savefig("{}/{}.png".format(self.logfolder, epoch))
                plt.close()

                img_list.append(img_grid_fake)

        return img_list, G_losses, D_losses 

    class Discriminator(nn.Module):
        def __init__(self, channels_img, features_d):
            super(DCGAN.Discriminator, self).__init__()
            self.disc = nn.Sequential(
                # input: N x channels_img x 64 x 64
                nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
                nn.LeakyReLU(0.2),
                # _block(in_channels, out_channels, kernel_size, stride, padding)
                self._block(features_d, features_d * 2, 4, 2, 1),
                self._block(features_d * 2, features_d * 4, 4, 2, 1),
                self._block(features_d * 4, features_d * 8, 4, 2, 1),
                # After all _block img output is 4x4 (Conv2d below makes into 1x1)
                nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
                nn.Sigmoid(),
            )

        def _block(self, in_channels, out_channels, kernel_size, stride, padding):
            return nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride,
                    padding,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2),
            )

        def forward(self, x):
            return self.disc(x)


    class Generator(nn.Module):
        def __init__(self, channels_noise, channels_img, features_g):
            super(DCGAN.Generator, self).__init__()
            self.net = nn.Sequential(
                # Input: N x channels_noise x 1 x 1
                self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
                self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
                self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
                self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
                nn.ConvTranspose2d(
                    features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
                ),
                # Output: N x channels_img x 64 x 64
                nn.Tanh(),
            )

        def _block(self, in_channels, out_channels, kernel_size, stride, padding):
            return nn.Sequential(
                nn.ConvTranspose2d(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride,
                    padding,
                    bias=False,
                ),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
            )

        def forward(self, x):
            return self.net(x)

In [ ]:
collection = "Collection1"
num_epochs = 1000
!mkdir $collection

In [ ]:
modelDCGAN = DCGAN('../../images/png/{}'.format(collection), collection, num_epochs=num_epochs)

In [ ]:
img_list, G_losses, D_losses = modelDCGAN.train()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig('{}/loss.png'.format(collection))
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list[::10]]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)
HTML(ani.to_jshtml())

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(modelDCGAN.dataloader))

In [ ]:
# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(modelDCGAN.device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.savefig('{}/result.png'.format(collection))
plt.show()

In [ ]:
with open('{}/gen'.format(collection), 'wb') as f:
    pickle.dump(modelDCGAN.gen, f)
with open('{}/disc'.format(collection), 'wb') as f:
    pickle.dump(modelDCGAN.disc, f)

In [ ]:
### CLIENT ###
with open('{}/gen'.format(collection), 'rb') as f:
    gen = pickle.load(f)
with torch.no_grad():
    no_grad_fake = gen(torch.randn(64, 100, 1, 1, device=torch.device("cuda" if (torch.cuda.is_available() and 1 > 0) else "cpu"))).detach().cpu()
img_grid_fake = vutils.make_grid(no_grad_fake[:1], normalize=True)
save_image(img_grid_fake, 'tmp.png')
img = cv2.imread('tmp.png', cv2.IMREAD_UNCHANGED)
width = 150
height = 150
dim = (width, height)
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
cv2.imwrite('tmp.png', resized)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(np.transpose(img_grid_fake,(1,2,0)))
plt.show()
plt.close()